# **Import libraries**

In [1]:
!pip install torch transformers nltk

In [ ]:
import pandas as pd
import numpy as np
from transformers import pipeline
import torch
import nltk


# **load model**

In [2]:


def load_summarizer(model_name="facebook/bart-large-cnn"):
    """
    Load the summarization model
    """
    print("Loading summarization model...")

    # Check if GPU is available
    device = 0 if torch.cuda.is_available() else -1
    print(f"Using {'GPU' if device == 0 else 'CPU'}")

    # Create summarization pipeline
    summarizer = pipeline(
        "summarization",
        model=model_name,
        device=device
    )

    print("Model loaded successfully!")
    return summarizer

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:82: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch_xla/experimental/gru.py:113: SyntaxWarning: invalid escape sequence '\_'
  * **h_n**: tensor of shape :math:`(D * \text{num\_layers}, H_{out})` or


In [3]:

# Download required NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except:
    nltk.download('punkt')

def process_text(text, max_length=1024):
    """
    Clean and prepare text for summarization
    """
    # Remove extra whitespace
    clean_text = text.strip()

    # If text is too long, shorten it
    if len(clean_text) > max_length:
        sentences = nltk.sent_tokenize(clean_text)
        shortened_text = ""

        # Add sentences until we reach the limit
        for sentence in sentences:
            if len(shortened_text + sentence) < max_length:
                shortened_text += sentence + " "
            else:
                break

        clean_text = shortened_text.strip()
        print(f"Text shortened to {len(clean_text)} characters")

    return clean_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# **Evaluate model**

In [5]:

class SimpleSummarizer:
    def __init__(self, model_name="facebook/bart-large-cnn"):
        self.summarizer = load_summarizer(model_name)

    def summarize(self, text, max_length=130, min_length=30):
        """
        Main function to summarize text
        """
        # Process the text
        processed_text = process_text(text)

        # Generate summary
        try:
            result = self.summarizer(
                processed_text,
                max_length=max_length,
                min_length=min_length,
                do_sample=False
            )
            return result[0]['summary_text']
        except Exception as e:
            return f"Error: {e}"

# **Train and show Results**

In [7]:
# from summarizer import SimpleSummarizer

def show_examples():
    """
    Show some example summaries
    """
    # Sample texts
    examples = [
        {
            "title": "AI Technology",
            "text": """
            Artificial intelligence is transforming many industries. Machine learning
            algorithms can now recognize patterns in data that humans might miss.
            Companies use AI for customer service, healthcare, and transportation.
            While AI offers many benefits, there are also concerns about job displacement
            and ethical issues that need to be addressed.
            """
        },
        {
            "title": "Climate Change",
            "text": """
            Climate change is causing significant changes to our planet. Scientists
            report rising global temperatures and more extreme weather events.
            Many countries are working to reduce carbon emissions and switch to
            renewable energy sources like solar and wind power.
            """
        }
    ]

    # Create summarizer
    summarizer = SimpleSummarizer()

    print("=== TEXT SUMMARIZATION EXAMPLES ===")
    print()

    for example in examples:
        print(f"📝 {example['title']}")
        print("Original text:")
        print(example['text'])
        print()

        summary = summarizer.summarize(example['text'])

        print("📋 Summary:")
        print(summary)
        print("\n" + "="*50 + "\n")

if __name__ == "__main__":
    show_examples()

Loading summarization model...
Using CPU


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Your max_length is set to 130, but your input_length is only 105. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=52)


Model loaded successfully!
=== TEXT SUMMARIZATION EXAMPLES ===

📝 AI Technology
Original text:

            Artificial intelligence is transforming many industries. Machine learning
            algorithms can now recognize patterns in data that humans might miss.
            Companies use AI for customer service, healthcare, and transportation.
            While AI offers many benefits, there are also concerns about job displacement
            and ethical issues that need to be addressed.
            



Your max_length is set to 130, but your input_length is only 79. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)


📋 Summary:
Artificial intelligence is transforming many industries. Companies use AI for customer service, healthcare, and transportation. While AI offers many benefits, there are also concerns about job displacement.


📝 Climate Change
Original text:

            Climate change is causing significant changes to our planet. Scientists
            report rising global temperatures and more extreme weather events.
            Many countries are working to reduce carbon emissions and switch to
            renewable energy sources like solar and wind power.
            

📋 Summary:
Climate change is causing significant changes to our planet. Scientists report rising global temperatures and more extreme weather events. Many countries are working to reduce carbon emissions and switch to renewable energy.




# **Generating text**

In [10]:
# from summarizer import SimpleSummarizer

def interactive_mode():
    """
    Interactive mode for user input
    """
    print("🎯 INTERACTIVE TEXT SUMMARIZER")
    print("Type 'quit' to exit\n")

    summarizer = SimpleSummarizer()

    while True:
        # Get user input
        text = input("Enter text to summarize:\n> ")

        if text.lower() == 'quit':
            print("Goodbye!")
            break

        if text.strip():
            print("\n⏳ Generating summary...")
            summary = summarizer.summarize(text)

            print("\n📋 SUMMARY:")
            print(summary)
            print("\n" + "="*60 + "\n")
        else:
            print("Please enter some text!\n")

if __name__ == "__main__":
    interactive_mode()

🎯 INTERACTIVE TEXT SUMMARIZER
Type 'quit' to exit

Loading summarization model...
Using CPU


Device set to use cpu


Model loaded successfully!
Enter text to summarize:
> i go to school by bus and i live in october and i love my kids


Your max_length is set to 130, but your input_length is only 19. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)



⏳ Generating summary...

📋 SUMMARY:
i live in october and i love my kids. i go to school by bus and i live in October. i live with my husband and my three children. i am a single mother of three.


Enter text to summarize:
> quit
Goodbye!


# **Test the model with the following text:**

In [13]:
# from demo import show_examples
# from interactive import interactive_mode

def main():
    """
    Main menu
    """
    while True:
        print("\n📖 TEXT SUMMARIZATION TOOL")
        print("1. See examples")
        print("2. Interactive mode")
        print("3. Exit")

        choice = input("\nChoose an option (1-3): ")

        if choice == '1':
            show_examples()
        elif choice == '2':
            interactive_mode()
        elif choice == '3':
            print("Goodbye!")
            break
        else:
            print("Please enter 1, 2, or 3")

if __name__ == "__main__":
    main()


📖 TEXT SUMMARIZATION TOOL
1. See examples
2. Interactive mode
3. Exit

Choose an option (1-3): 1
Loading summarization model...
Using CPU


Device set to use cpu
Your max_length is set to 130, but your input_length is only 105. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=52)


Model loaded successfully!
=== TEXT SUMMARIZATION EXAMPLES ===

📝 AI Technology
Original text:

            Artificial intelligence is transforming many industries. Machine learning
            algorithms can now recognize patterns in data that humans might miss.
            Companies use AI for customer service, healthcare, and transportation.
            While AI offers many benefits, there are also concerns about job displacement
            and ethical issues that need to be addressed.
            



Your max_length is set to 130, but your input_length is only 79. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)


📋 Summary:
Artificial intelligence is transforming many industries. Companies use AI for customer service, healthcare, and transportation. While AI offers many benefits, there are also concerns about job displacement.


📝 Climate Change
Original text:

            Climate change is causing significant changes to our planet. Scientists
            report rising global temperatures and more extreme weather events.
            Many countries are working to reduce carbon emissions and switch to
            renewable energy sources like solar and wind power.
            

📋 Summary:
Climate change is causing significant changes to our planet. Scientists report rising global temperatures and more extreme weather events. Many countries are working to reduce carbon emissions and switch to renewable energy.



📖 TEXT SUMMARIZATION TOOL
1. See examples
2. Interactive mode
3. Exit

Choose an option (1-3): 3
Goodbye!
